In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from tensorflow.keras.preprocessing import image
import os

### Load Model from weights

In [2]:
IMAGE_SHAPE = (224,224,3)

In [3]:
model = tf.keras.models.load_model('model_transfer.h5')


### Detect with test data

In [11]:
def detect_image(image_dir, name_file):
    img = image.load_img(image_dir, target_size=(224,224))
    #imgplot = plt.imshow(img)
    x = image.img_to_array(img) 
    x = np.expand_dims(x, axis=0)
    
    class_detect = 'None'
    
    threshold = 0.6
    images = np.vstack([x])
    classes = model.predict(images, batch_size=10)
    result = np.argmax(classes)
    confident = classes[0][result]
    
    if threshold < confident:
        if result == 0:
            class_detect = 'fire'
        else:
            class_detect = 'Smoke'

    print('{} detected as {} with confident score = {}'.format(name_file, class_detect, confident))


In [12]:
list_test = os.listdir('test data/')
for file in list_test:
    detect_image('test data/'+file, file)

fire.jpg detected as fire with confident score = 0.9307780265808105
fire2.jpg detected as fire with confident score = 0.9875051975250244
fire3.jpg detected as Smoke with confident score = 0.743914008140564
fire4.jpg detected as Smoke with confident score = 0.8429162502288818
fire5.jpg detected as fire with confident score = 0.9525253772735596
smoke.jpg detected as Smoke with confident score = 0.6671893000602722
smoke2.jpg detected as fire with confident score = 0.8811296224594116
smoke3.jpg detected as Smoke with confident score = 0.9571495652198792
smoke4.jpg detected as None with confident score = 0.5033536553382874
smoke5.jpg detected as Smoke with confident score = 0.6567634344100952


### Test with video

In [13]:
!pip install opencv-python

In [21]:
import cv2

src = cv2.VideoCapture('video.mp4')
color_dict={0:(0,255,0),1:(0,0,255)}
class_detect = 'None'
threshold = 0.7
font = cv2.FONT_HERSHEY_SIMPLEX
while(True):
    ret,frame = src.read()
    
    img = cv2.resize(frame,(224,224))     
    img = img.reshape(1,224,224,3)
    
    images = np.vstack([img])
    
    classes = model.predict(images, batch_size=10)
    result = np.argmax(classes)
    confident = classes[0][result]
    
    if threshold < confident:
        if result == 0:
            class_detect = 'fire'
        else:
            class_detect = 'Smoke'
    else:
        class_detect = ''
        
    cv2.putText(frame, 
                class_detect, 
                (50, 50), 
                font, 1, 
                (0, 255, 255), 
                2, 
                cv2.LINE_4)

    cv2.imshow("LIVE", frame)
    key =cv2.waitKey(1)

    if(key==27):
        break

cv2.destroyAllWindows()
src.release()